In [5]:
import os
from dotenv import load_dotenv
load_dotenv() ### loading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [8]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [9]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Jayanth and I am an AI Engineer")])

AIMessage(content="Hello Jayanth, it's nice to meet you!\n\nAs an AI, I don't have personal experiences or feelings, but I'm always eager to learn and help. \n\nWhat can I do for you today? Are you working on any interesting projects?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 22, 'total_tokens': 81, 'completion_time': 0.107272727, 'prompt_time': 0.000140239, 'queue_time': 0.019504361, 'total_time': 0.107412966}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2a35892f-5c6f-4f36-aab8-f2c4b289af76-0', usage_metadata={'input_tokens': 22, 'output_tokens': 59, 'total_tokens': 81})

In [10]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Jayanth and I am a  AI Engineer"),
        AIMessage(content="Hello Jayanth, it's nice to meet you!\n\nAs an AI, I don't have personal experiences or feelings, but I'm always eager to learn and help. \n\nWhat can I do for you today?"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="According to our conversation, your name is Jayanth and you are an AI Engineer.   \n\nIs there anything else you'd like to know or discuss? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 93, 'total_tokens': 130, 'completion_time': 0.067272727, 'prompt_time': 0.00433644, 'queue_time': 0.02106947, 'total_time': 0.071609167}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e857538-8108-45c5-ab12-e8d8fe62a930-0', usage_metadata={'input_tokens': 93, 'output_tokens': 37, 'total_tokens': 130})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [12]:
config={"configurable":{"session_id":"chat1"}}

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Jayanth and I am a AI Engineer")],
    config=config
)

In [15]:
response.content

"Hello Jayanth, it's nice to meet you! \n\nAs an AI, I don't have personal experiences or feelings, but I'm always ready to help with any AI-related questions or tasks you might have. \n\nWhat can I do for you today? 😄  \n\n"

In [16]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Jayanth, as you told me earlier. 😊  \n\n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 99, 'total_tokens': 127, 'completion_time': 0.050909091, 'prompt_time': 0.002504355, 'queue_time': 0.020801433, 'total_time': 0.053413446}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bb8ae7be-6a5f-4142-bad5-d6d1a4f0be5e-0', usage_metadata={'input_tokens': 99, 'output_tokens': 28, 'total_tokens': 127})

In [17]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊  \n\n"

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Naman")],
    config=config1
)
response.content

"Hi Naman! It's nice to meet you. 👋\n\nWhat can I do for you today? 😊  \n"

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Naman!  😊  I remember that from our last conversation. \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [20]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [21]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hi Krish! It's nice to meet you. 👋 \n\nWhat can I do for you today? I'm ready to answer your questions to the best of my ability. 😊  Just ask away! \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 30, 'total_tokens': 77, 'completion_time': 0.085454545, 'prompt_time': 0.000140499, 'queue_time': 0.021295619000000002, 'total_time': 0.085595044}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e0e72db7-136a-422a-ab5a-6f89a176cd9f-0', usage_metadata={'input_tokens': 30, 'output_tokens': 47, 'total_tokens': 77})

In [22]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [24]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Jayanth")],
    config=config
)

response

AIMessage(content="Hi Jayanth!  \n\nIt's nice to meet you too!  What can I do for you? 😄\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 71, 'total_tokens': 98, 'completion_time': 0.049090909, 'prompt_time': 0.010012034, 'queue_time': 0.019296773, 'total_time': 0.059102943}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ee673e46-ff58-478e-82cf-351ffb207e02-0', usage_metadata={'input_tokens': 71, 'output_tokens': 27, 'total_tokens': 98})

In [25]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jayanth. 😊 \n\nIs there anything else I can help you with?\n'

In [26]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [27]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Jayanth")],"language":"Telugu"})
response.content

'నమస్తే జయంత్, నేను మీకు సహాయకరి అని చెప్పండి. ఏదైనా ప్రశ్న ఉంటే, నేను నా శక్తి సరికొలది సమాధానం ఇచ్చేందుకు సిద్ధంగా ఉన్నాను. 😊\n\n'

In [28]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [29]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Jayanth")],"language":"Telugu"},
    config=config
)
repsonse.content

'నమస్తే జయంత్! 😊 \n\nమీకు ఎలా సహాయం చేయగలను? \n'

In [30]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Telugu"},
    config=config,
)

In [31]:
response.content


'మీ పేరు జయంత్. 😉 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [32]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

e:\MachineLearning_Course\GenAI\Practice_Projects\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\MachineLearning_Course\GenAI\Practice_Projects\venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jayan\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In or

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [33]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your taste in ice cream! \n\nWhat's your favorite flavor?  🍦 😊\n"

In [34]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked: "what is 2 + 2" \n\n\nLet me know if you\'d like to try another one! 😊  \n'

In [35]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [36]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  Would you like to tell me your name? 😊 \n"

In [37]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊 \n\n"